In [ ]:
# RDF

import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt

# Load your topology and trajectory files
u = mda.Universe('topology', 'trajectory')

protein_ligand = u.select_atoms('protein') # select protein atoms
water = u.select_atoms('resname SOL')      # select water

from MDAnalysis.analysis import rdf
rdf_analysis = rdf.InterRDF(protein_ligand, water, nbins=100, range=(0.0, 10.0))
rdf_analysis.run()

plt.figure(figsize=(10, 7))
plt.plot(rdf_analysis.bins, rdf_analysis.rdf)
plt.xlim(1.0, 4)
plt.ylim(0, 0.5)
plt.xlabel('Distance (Å)')
plt.ylabel('g(r)')
plt.legend()
plt.show()
plt.savefig('rdf.png', dpi=500)


In [ ]:
# Average water residence time

import numpy as np
import matplotlib.pyplot as plt

def calculate_residence_time(protein_ligand, water, cutoff=3.5):
    residence_times = []

    for ts in u.trajectory:
        # Calculate distances between protein-ligand and water molecules
        distances = np.sqrt(((protein_ligand.positions[:, np.newaxis, :] - water.positions[np.newaxis, :, :])**2).sum(axis=2))
        
       
        within_cutoff = distances < cutoff
        
        # Calculate the number of water molecules within cutoff for each protein-ligand
        residence_time = np.sum(within_cutoff, axis=1)
        residence_times.append(residence_time)
    
    # Convert list to numpy array for easier manipulation
    residence_times = np.array(residence_times)
    
    # Return the mean residence time over all frames
    return np.mean(residence_times, axis=0)


residence_times_frames = calculate_residence_time(protein_ligand, water)


time_step_ps = 10  # Change this to actual time step

# Convert residence times from frames to picoseconds
residence_times_ps = residence_times_frames * time_step_ps

# Calculate average residence time in picoseconds
average_residence_time_ps = np.mean(residence_times_ps)


plt.figure(figsize=(10, 7))
plt.hist(residence_times_ps, bins=100, edgecolor='black', alpha=0.7)


plt.axvline(average_residence_time_ps, color='red', linestyle='dashed', linewidth=1.5, label=f'Average Residence Time = {average_residence_time_ps:.2f} ps')

plt.xlabel('Residence Time (ps)')
plt.ylabel('Frequency')
#plt.title('Residence Time Distribution of Water Molecules')
plt.legend()
plt.grid(True)


plt.savefig('res-time.png', dpi=500)


plt.show()


print(f'Average Residence Time: {average_residence_time_ps:.2f} ps')


In [ ]:
# Solvent density

import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.ndimage import gaussian_filter

# Load your trajectory and topology files
trajectory_file = 'trajectory'  # Replace with trajectory file .xtc
topology_file = 'topology'         # Replace with topology file .tpr/.gro

u = mda.Universe(topology_file, trajectory_file)

# Select the ligand and solvent (water) atoms
ligand_selection = 'resname UNL'  # Replace with ligand residue name
solvent_selection = 'resname SOL and name OW'  # Replace with your solvent/residue name

ligand_atoms = u.select_atoms(ligand_selection)
solvent_atoms = u.select_atoms(solvent_selection)
ligand_com = np.array([ligand_atoms.center_of_mass() for ts in u.trajectory])

# Define the size and resolution of the 3D grid (e.g., 0.5 Å grid spacing)
grid_resolution = 0.5  # Grid spacing in Å
grid_size = 10  # Grid size in Å (the region where we will calculate the distribution)

# Create a 3D grid to calculate the solvent density
x_bins = np.arange(-grid_size, grid_size + grid_resolution, grid_resolution)
y_bins = np.arange(-grid_size, grid_size + grid_resolution, grid_resolution)
z_bins = np.arange(-grid_size, grid_size + grid_resolution, grid_resolution)

# Initialize the 3D grid for solvent density (SDF)
solvent_density = np.zeros((len(x_bins) - 1, len(y_bins) - 1, len(z_bins) - 1))

for ts in u.trajectory:
    solvent_positions = solvent_atoms.positions

    distances = np.linalg.norm(solvent_positions - ligand_com[ts.frame], axis=1)
    
    cutoff_distance = 5.0
    solvent_within_cutoff = solvent_positions[distances <= cutoff_distance]

    for atom_position in solvent_within_cutoff:
        relative_position = atom_position - ligand_com[ts.frame]
        
        # Convert to grid coordinates (x, y, z)
        x_idx = np.digitize(relative_position[0], x_bins) - 1
        y_idx = np.digitize(relative_position[1], y_bins) - 1
        z_idx = np.digitize(relative_position[2], z_bins) - 1

        if 0 <= x_idx < len(x_bins) - 1 and 0 <= y_idx < len(y_bins) - 1 and 0 <= z_idx < len(z_bins) - 1:
            solvent_density[x_idx, y_idx, z_idx] += 1

solvent_density_smooth = gaussian_filter(solvent_density, sigma=1)

# Create a meshgrid for the 3D grid coordinates
x_center = (x_bins[:-1] + x_bins[1:]) / 2
y_center = (y_bins[:-1] + y_bins[1:]) / 2
z_center = (z_bins[:-1] + z_bins[1:]) / 2

slice_index = len(z_center) // 2  # Choose a slice at the center of the grid 

density_slice = solvent_density_smooth[:, :, slice_index]

X, Y = np.meshgrid(x_center, y_center)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')


surf = ax.plot_surface(X, Y, density_slice, cmap='PiYG', edgecolor='none', alpha=0.7)

ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.set_zlabel('Density')
#ax.set_title('Smooth 3D Spatial Distribution Function (SDF) Around Ligand')

#ax.set_zlim(0, 50)

cbar = fig.colorbar(surf)
cbar.set_label('Solvent Density')

plt.show()
fig.savefig('solvent_density_plot.png', dpi=300, bbox_inches='tight')


In [ ]:
# Orientational Order Parameter

import numpy as np
import MDAnalysis as mda
from MDAnalysis.lib.distances import distance_array
import pandas as pd
import matplotlib.pyplot as plt

def compute_q(center, neighbors):
    vecs = neighbors - center
    vecs /= np.linalg.norm(vecs, axis=1)[:, None]
    q = 0.0
    for j in range(3):
        for k in range(j + 1, 4):
            cos_psi = np.dot(vecs[j], vecs[k])
            q += (cos_psi + 1.0 / 3.0) ** 2
    return 1.0 - (3.0 / 8.0) * q

def get_water_oop_around_ligand(topology, trajectory, cutoff=3.5,
                                 output_csv="oop_data.csv",
                                 output_plot="oop_plot.png"):
    u = mda.Universe(topology, trajectory)
    
    # Adjust these to match your system
    water_oxygens = u.select_atoms("resname SOL and name OW")
    ligand = u.select_atoms("resname UNL")
    
    if len(water_oxygens) == 0:
        raise ValueError("No water oxygens found. Check selection string.")
    if len(ligand) == 0:
        raise ValueError("No ligand atoms found. Check resname.")

    results = []

    for ts in u.trajectory:
        lig_coords = ligand.positions
        water_coords = water_oxygens.positions

        dists = distance_array(water_coords, lig_coords)
        close_idxs = np.where(np.min(dists, axis=1) < cutoff)[0]
        waters_in_shell = water_oxygens[close_idxs]

        frame_qs = []

        for water in waters_in_shell:
            neighbor_dists = np.linalg.norm(water_coords - water.position, axis=1)
            sorted_idxs = np.argsort(neighbor_dists)
            neighbor_idxs = [i for i in sorted_idxs if not np.allclose(water_coords[i], water.position)][0:4]
            if len(neighbor_idxs) < 4:
                continue
            neighbors = water_coords[neighbor_idxs]
            q = compute_q(water.position, neighbors)
            frame_qs.append(q)

        avg_q = np.mean(frame_qs) if frame_qs else np.nan
        time_ps = ts.time
        results.append([ts.frame, time_ps, avg_q])
#        print(f"Frame {ts.frame}, Time {time_ps:.1f} ps: avg q = {avg_q:.4f}")

    df = pd.DataFrame(results, columns=["Frame", "Time_ps", "Average_OOP_q"])
    df.to_csv(output_csv, index=False)
    print(f"\nSaved data to {output_csv}")

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(df["Time_ps"], df["Average_OOP_q"], label="Average q", color="blue")
    plt.xlabel("Time (ps)")
    plt.ylabel("Average OOP (q)")
#    plt.title("Tetrahedral Order Parameter (q) Around Ligand Over Time")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(output_plot, dpi=300)
    print(f"Saved plot to {output_plot}")

    get_water_oop_around_ligand("topology", "trajectory", cutoff=4.0)
